In [3]:
import os
import sys
import numpy as np

In [13]:
class genAmorphous:
    def __init__(self, 
                 density, 
                 chem_formula,
                 outfile = 'POSCAR',
                 clear_dummy = True):
        """
        Arg 1: (float) density in g/cm3. (ex. 10.00)\n
        Arg 2: (str) chemical forumula. (ex. Hf32O64Ag1)\n
        Arg 3: (str) name of POSCAR. (default:POSCAR)\n
        Arg 4: (bool) remove dummy files. (default:True)
        """
        self.density = density
        self.chem_formula = chem_formula

        self.atomMass = {}
        self.read_atomic_mass_table()

        self.atom_name = []
        self.atom_num = []
        self.divide_chem_formula()

        # check atom name
        for name in self.atom_name:
            if not(name in self.atomMass.keys()):
                print(f"{name} is unknown atom.")
                sys.exit(0)

        self.alat = None
        self.alat_pbc = None
        self.get_dimension_cubic()

        self.write_pdb()
        self.write_packmol_input()

        # run packmol
        os.system("packmol < packmol.inp > packmol.out")

        # write POSCAR
        self.covert_pdb_to_poscar(outfile)

        if clear_dummy:
            self.clear_dummy_files()

    def read_atomic_mass_table(self):
        if not(os.path.isfile('atomic_mass_table.dat')):
            print('atomic_mass_table.dat is not found.\n \
                  please visit https://github.com/TY-Jeong/arpaca')
            sys.exit(0)
        lines = [line.strip() for line in open('atomic_mass_table.dat')]
        for line in lines:
            [name, mass] = line.split('\t')
            self.atomMass[name] = float(mass)

    def divide_chem_formula(self):
        check, word = 'alpha', ''
        for letter in self.chem_formula:
            if letter.isalpha():
                if check == 'alpha':
                    word += letter
                elif check == 'num':
                    self.atom_num += [int(word)]
                    check = 'alpha'
                    word = letter
            else:
                if check == 'alpha':
                    self.atom_name += [word]
                    check = 'num'
                    word = letter
                elif check == 'num':
                    word += letter
        self.atom_num += [int(word)]
        self.atom_num = np.array(self.atom_num)

    def get_dimension_cubic(self):
        u  =  1.660539e-24 # g
        mass = [self.atomMass[name]*u for name in self.atom_name]
        mass = np.array(mass)
        m = np.sum(mass * self.atom_num)
        self.alat = (m/self.density)**(1/3)*1e8  # Å
        self.alat_pbc = self.alat*0.98 - 2

    def write_pdb(self):
        for name in self.atom_name:
            with open(f"./{name}.pdb",'w') as f:
                f.write(f"COMPND    {name}_atom\n")
                f.write(f"COMPND   1Created by aRPaCa\n")
                f.write("HETATM    1 %-2s           1       5.000   5.000   5.000\n"%(name))
                f.write("END")

    def write_packmol_input(self):
        with open("./packmol.inp",'w') as f:
            f.write("tolerance 2.0\n")
            f.write("filetype pdb\n")
            f.write("output packmol_output.pdb\n\n")

            for name, num in zip(self.atom_name,self.atom_num):
                f.write(f"structure {name}.pdb\n")
                f.write(f"  number {num}\n")
                f.write("  inside cube 0. 0. 0. %.6f \n"%self.alat_pbc)
                f.write("end structure\n\n")

    def covert_pdb_to_poscar(self, outfile):
        lines = [line.strip() for line in open('./packmol_output.pdb')]
        num_tot = np.sum(self.atom_num)
        shift = (self.alat-self.alat_pbc)/2
        with open(outfile, 'w') as f:
            f.write("Generated by aRPaCa\n")
            f.write("1.0\n")
            f.write("   %.6f      0.000000      0.000000\n"%self.alat)
            f.write("   0.000000      %.6f      0.000000\n"%self.alat)
            f.write("   0.000000      0.000000      %.6f\n"%self.alat)
            for name in self.atom_name:
                f.write("   "+name)
            f.write("\n")
            for num in self.atom_num:
                f.write("   "+str(num))
            f.write("\n")
            f.write("Cartesian\n")
            for line in lines[5:5+num_tot]:
                coord = np.array(line.split()[-3:], dtype=float)
                coord += np.array([shift, shift, shift])
                f.write("    %.6f    %.6f   %.6f\n"%(coord[0], coord[1], coord[2]))
    
    def clear_dummy_files(self):
        for name in self.atom_name:
            os.remove(f"{name}.pdb")
        os.remove('packmol.inp')
        os.remove('packmol.out')
        os.remove('packmol_output.pdb')

In [38]:
class genInput:
    def __init__(self,
                 potcar = 'pbe',
                 nsw = 10000,
                 potim = 2,
                 temp = 4000,
                 charge = 0,
                 ncore = 4):
        """
        Arg 1: (str) kind of potcar: lda or pbe (default:pbe)\n
        Arg 2: (int) iteration number. (default:10000)\n
        Arg 3: (int) time step in fs. (default: 2)\n
        Arg 4: (float) temperature in K (default: 4000)\n
        Arg 5: (float) charge state. (ex. 2 for +2) (default: 0)\n
        Arg 6: (int) number of core. (default: 4)
        """
        self.potcar = potcar
        self.nsw = nsw
        self.potim = potim
        self.temp = temp
        self.charge = charge
        self.ncore = ncore

        # write KPOINTS
        self.write_kpoints()
        
        # write POTCAR
        self.pot_recommend = {}
        self.read_recommended_potcar()

        self.prefix_pot = None
        self.read_path_potcar()

        self.atom_name = None
        self.atom_num = None
        self.zval =[]
        self.enmax = []
        self.write_potcar()

        # write INCAR
        self.atom_num = np.array(self.atom_num)
        self.zval = np.array(self.zval)
        self.nelect = np.sum(self.atom_num*self.zval)-self.charge

        self.write_incar()

    def read_recommended_potcar(self):
        if not(os.path.isfile('recommended_potcar.dat')):
            print('recommended_potcar.dat is not found.\n \
                  please visit https://github.com/TY-Jeong/arpaca')
            sys.exit(0)
        lines = [line.strip() for line in open('recommended_potcar.dat')]
        for line in lines:
            [name, recommend] = line.split('\t')
            self.pot_recommend[name] = recommend
    
    def read_path_potcar(self):
        if not(os.path.isfile('path_potcar.dat')):
            print('path_potcar.dat is not found.\n')
            sys.exit(0)
        lines = [line.strip() for line in open('path_potcar.dat')]
        self.prefix_pot = lines[0].split('=')[1] if self.potcar == 'pbe'\
            else lines[1].split('=')[1]
        if self.prefix_pot[-1] != '/':
            self.prefix_pot += '/'

    def write_kpoints(self):
        with open('./KPOINTS','w') as f:
            f.write("kpoints\n")
            f.write("0\n")
            f.write("G\n")
            f.write("1 1 1\n")
            f.write("0 0 0")
            
    def write_potcar(self):
        if not(os.path.isfile('POSCAR')):
            print('no POSCAR file exists.')
            sys.exit(0)
        lines = [line.strip() for line in open('POSCAR')]
        self.atom_name = lines[5].split()
        self.atom_num = np.array(lines[6].split(), dtype=int)

        with open('./POTCAR','w') as f:
            for name in self.atom_name:
                pot = self.prefix_pot+self.pot_recommend[name]+"/POTCAR"
                lines = [line for line in open(pot)]
                for line in lines:
                    f.write(line)
                    if 'ZVAL' in line:
                        self.zval += [float(line.split()[5])]
                    if 'ENMAX' in line:
                        self.enmax += [float(line.split()[2].split(';')[0])]

    def write_incar(self):
        system = ''
        for name, num in zip(self.atom_name, self.atom_num):
            system += name+str(num)

        with open("./INCAR",'w') as f:
            f.write(f"SYSTEM = {system}\n")
            f.write(f"NCORE = {self.ncore}\n")
            f.write('\n')
            f.write('# electronic degrees\n')
            f.write(f"ENCUT = {np.max(self.enmax)}\n")
            f.write("LREAL = Auto\n")
            f.write("PREC  = Normal\n")
            f.write("EDIFF = 1E-5\n")
            f.write("ISMEAR = 0\n")
            f.write("SIGMA = 0.01\n")
            f.write("ALGO = Fast\n")
            f.write("MAXMIX = 40\n")
            f.write("ISYM = 0\n")
            f.write("NELMIN = 4\n")
            f.write('\n')
            f.write('# molecular dynamics\n')
            f.write('IBRION = 0\n')
            f.write(f"NSW = {self.nsw}\n")
            f.write(f"POTIM = {self.potim}\n")
            f.write("NWRITE = 0\n")
            f.write("NBLOCK = 1\n")
            f.write("LCHARG = .FALSE.\n")
            f.write("LWAVE = .FALSE.\n")
            f.write(f"TEBEG = {self.temp}\n")
            f.write(f"TEEND = {self.temp}\n")
            f.write('\n')
            f.write('# canonical (Nose-Hoover) thermostat\n')
            f.write("MDALGO = 2\n")
            f.write("SMASS =  0\n")
            f.write("ISIF = 2\n")
            f.write('\n')
            f.write('# charge state\n')
            f.write("NELECT = %.3f"%self.nelect)

In [59]:
class xdat2pos:
    def __init__(self, xdatcar, *args):
        """
        Arg 1: (str) XDATCAR file\n
        Arg 2: (case 1) step (for one POSCAR)\n
            (case 2) start, end, interval (for multiple POSCAR)
        """
        if os.path.isfile(xdatcar):
            self.xdatcar = xdatcar
        else:
            print(f"{xdatcar} is not found.")
            sys.exit(0)

        # read xdatcar
        self.lines = [line for line in open(xdatcar)]
        self.num_atoms = np.array(self.lines[6].split(), dtype=int).sum()

        if len(args) == 1:
            step = int(args[0])
            self.write_poscar(step)
        elif len(args) == 3:
            start, end, interval = int(args[0]), int(args[1]), int(args[2]) 
            steps = np.arange(start, end+1, interval)
            if not(os.path.isdir('poscars')):
                os.mkdir('poscars')
            for step in steps:
                self.write_poscar(step, prefix='poscars/')
        else:
            print("expected number of argumnets is 2 or 4.")
            sys.exit(0)
    
    def write_poscar(self, step, prefix='./'):
        check = False
        file_out = f"POSCAR_{step}"
        with open(prefix+file_out, 'w') as f:
            for num, line in enumerate(self.lines):
                if num < 6:
                    f.write(line)
                elif num == 6:
                    f.write(line)
                    f.write("Direct\n")
                else:
                    if "Direct" in line and " "+str(step)+"\n" in line:
                        for line in self.lines[num+1:num+1+self.num_atoms]:
                            f.write(line)
                        print(f"{file_out} was generated.")
                        check = True
        if not check:
            print("step is out of range.")
            os.remove(prefix+file_out)

In [57]:
struc = genAmorphous(10.00, 'Hf34O67')
inputs = genInput(charge=2)

POSCAR_1000 was generated.
POSCAR_2000 was generated.
POSCAR_3000 was generated.
POSCAR_4000 was generated.
POSCAR_5000 was generated.
POSCAR_6000 was generated.
POSCAR_7000 was generated.
POSCAR_8000 was generated.
POSCAR_9000 was generated.
POSCAR_10000 was generated.


In [56]:
posgen = xdat2pos('XDATCAR', 1000, 10000, 1000)

POSCAR_1000 was generated.
POSCAR_2000 was generated.
POSCAR_3000 was generated.
POSCAR_4000 was generated.
POSCAR_5000 was generated.
POSCAR_6000 was generated.
POSCAR_7000 was generated.
POSCAR_8000 was generated.
POSCAR_9000 was generated.
POSCAR_10000 was generated.
